In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [4]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [7]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [8]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [9]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [10]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

In [12]:
src_data[0]

tensor([ 831, 1089, 1544, 2858, 1944, 3100, 1948, 2870, 4983,  325, 2870, 3484,
        4936,  776,  413, 3564, 1520, 2668, 4321,  634, 4942, 4309, 4283, 2581,
        1289, 4999, 2518, 3404, 1188, 3217, 1015, 2550, 1674, 4367, 2944, 4626,
        1230, 2217, 4564, 4867, 4028, 2351, 2622, 4405,  641, 2619,  268, 1996,
        3703, 1517, 4098, 2349, 1367,   70, 1559, 2460, 2731, 3183, 3785,  774,
        2744, 1755, 2758, 1482, 3119, 1522, 3408,  622, 3112, 3507, 3140,  333,
        2731, 2133,  717, 1693, 2968, 4518, 4998, 3096, 3881, 1046, 3348, 1088,
        1930, 2789, 4578, 1672,  328, 2147, 2390, 1691, 4629, 4033, 4193, 2040,
        3452, 2402, 3028,  498])

In [13]:
tgt_data[0]

tensor([2312, 1024, 1319,   97, 1381, 2151, 2639, 4886, 2458, 4039, 4577, 4275,
        1947,  187, 3913, 3555,   71, 4783, 1219,  895, 3520,  586, 4693, 1868,
        1381, 1262, 1728, 2946, 1513, 4065, 2613, 3969, 4674, 1468, 2524, 2009,
        1706, 1708, 1464, 4197,  422, 4722, 2695, 4799, 2380, 3196, 1168, 3928,
        3908, 1153, 2662, 4355, 1955, 1335, 1165, 2539, 1474, 4729, 1511, 3915,
         158,  590, 2148, 3452,  693,  703, 4104,  736, 3649,  330, 1075,  281,
         384, 3299, 2420, 2458, 1591, 1262, 4084, 2937, 1907, 1585, 4443,   44,
        1661,  957, 1746,  751, 2732, 1514, 1424, 2700, 3026, 1703, 1206, 3251,
        4677, 3646, 4407, 4759])

In [14]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.685395240783691
Epoch: 2, Loss: 8.54706859588623
Epoch: 3, Loss: 8.47192096710205
Epoch: 4, Loss: 8.41326904296875
Epoch: 5, Loss: 8.356627464294434
Epoch: 6, Loss: 8.285500526428223
Epoch: 7, Loss: 8.206507682800293
Epoch: 8, Loss: 8.125670433044434
Epoch: 9, Loss: 8.04018783569336
Epoch: 10, Loss: 7.955439567565918
Epoch: 11, Loss: 7.878020763397217
Epoch: 12, Loss: 7.7993621826171875
Epoch: 13, Loss: 7.710813045501709
Epoch: 14, Loss: 7.63084602355957
Epoch: 15, Loss: 7.545557975769043
Epoch: 16, Loss: 7.4593424797058105
Epoch: 17, Loss: 7.379349708557129
Epoch: 18, Loss: 7.298091411590576
Epoch: 19, Loss: 7.219499111175537
Epoch: 20, Loss: 7.137020111083984
Epoch: 21, Loss: 7.053585052490234
Epoch: 22, Loss: 6.981748580932617
Epoch: 23, Loss: 6.8979363441467285
Epoch: 24, Loss: 6.828252792358398
Epoch: 25, Loss: 6.753929615020752
Epoch: 26, Loss: 6.6818928718566895
Epoch: 27, Loss: 6.597039222717285
Epoch: 28, Loss: 6.535000801086426
Epoch: 29, Loss: 6.45879077911